# Split complex task 



## Chain-of-thought (CoT) prompting 
Introduced in Wei et al. (2022)(opens in a new tab), `chain-of-thought (CoT)` prompting enables complex reasoning capabilities through intermediate reasoning steps. You can combine it with few-shot prompting to get better results on more complex tasks that require reasoning before responding.

### 1. Logic challenge


In [ ]:
#r "nuget: Azure.AI.OpenAI, 1.0.0-beta.5"

using Azure;
using Azure.AI.OpenAI;
using static System.Environment;

// Import the helper class to load configuration values
#!import ../Config/TheHelper.cs

// Load configuration values
string endpoint = TheHelper.GetConfigurationValue("endpoint");
string apiKey = TheHelper.GetConfigurationValue("apikey");
string deploymentName =TheHelper.GetConfigurationValue("DeploymentName");

// Create the OpenAI client
OpenAIClient client = new(new Uri(endpoint), new AzureKeyCredential(apiKey));

// Print a message to indicate that the client is ready to use
Console.WriteLine("OpenAI client is ready to use.");

This confusion prompt will generate a wrong or contradictory answer.

In [ ]:
// Create a new ChatCompletionsOptions object with a system message and a maximum token count
// For this example we are using ChatCompletionsOptions to send a prompt to the API and get a response
var chatCompletionsOptions = new ChatCompletionsOptions()
{
    Messages =
    {
        new ChatMessage(ChatRole.System, @"You are an AI assistant that helps to solve math calculations."),
    },
    MaxTokens = 100
};

// Set the chat prompt and add it to the ChatCompletionsOptions object as a user message
string thePrompt = @"The odd numbers in this group add up to an even number: 15, 32, 5, 13, 82, 7, 1. ";
chatCompletionsOptions.Messages.Add(new ChatMessage(ChatRole.User, thePrompt));

// Send the ChatCompletionsOptions object to the OpenAI API and get a response
Response<ChatCompletions> response = client.GetChatCompletions(deploymentOrModelName: deploymentName, chatCompletionsOptions);

// Print the chat response to the console
Console.WriteLine((response.Value.Choices[0].Message.Content));
Console.WriteLine();

Chain of Thought Prompting revolutionizes the conventional question-and-answer paradigm by compelling the model to generate intermediate reasoning steps prior to presenting the ultimate solution to a multi-step problem. The underlying principle is to mimic an intuitive thought process, wherein the model-produced Chain of Thought mirrors the logical progression one would undertake when tackling complex reasoning challenges. This methodology empowers the model to exhibit enhanced problem-solving capabilities and exhibit human-like reasoning abilities.

Let's assist the model by providing it with some samples, which will serve as a means of enhancing `its cognitive capabilities`.
By incorporating these intermediate steps within the prompt, we foster an environment that prompts the model to adhere to a meticulously structured thought process closely emulating the principles of human reasoning.

In [ ]:
// Create a new ChatCompletionsOptions object with a system message and a maximum token count
var chatCompletionsOptions = new ChatCompletionsOptions()
{
    Messages =
    {
        new ChatMessage(ChatRole.System, @"You are an AI assistant that helps to solve math calculations."),
        // All the following User messages and Assistant responses are exaples to help the model to generate the correct answer
        new ChatMessage(ChatRole.User, "The odd numbers in this group add up to an even number: 4, 8, 9, 15, 12, 2, 1."),
        new ChatMessage(ChatRole.Assistant, "Adding all the odd numbers (9, 15, 1) gives 25. The answer is False."),
        new ChatMessage(ChatRole.User, "The odd numbers in this group add up to an even number: 17,  10, 19, 4, 8, 12, 24."),
        new ChatMessage(ChatRole.Assistant, "Adding all the odd numbers (17, 19) gives 36. The answer is True."),
        new ChatMessage(ChatRole.User, "The odd numbers in this group add up to an even number: 16,  11, 14, 4, 8, 13, 24."),
        new ChatMessage(ChatRole.Assistant, "Adding all the odd numbers (11, 13) gives 24. The answer is True."),
        new ChatMessage(ChatRole.User, "The odd numbers in this group add up to an even number: 17,  9, 10, 12, 13, 4, 2."),
        new ChatMessage(ChatRole.Assistant, "Adding all the odd numbers (17, 9, 13) gives 39. The answer is False."),
    },
    MaxTokens = 100
};

// Set the chat prompt and add it to the ChatCompletionsOptions object as a user message
string thePrompt = @"The odd numbers in this group add up to an even number: 15, 32, 5, 13, 82, 7, 1. ";
chatCompletionsOptions.Messages.Add(new ChatMessage(ChatRole.User, thePrompt));

// Send the ChatCompletionsOptions object to the OpenAI API and get a response
Response<ChatCompletions> response = client.GetChatCompletions(deploymentOrModelName: deploymentName, chatCompletionsOptions);

// Print the chat response to the console
Console.WriteLine((response.Value.Choices[0].Message.Content));
Console.WriteLine();

## 2. Zero-shot COT Prompting
One recent idea that came out more recently is the idea of zero-shot CoT(opens in a new tab) (Kojima et al. 2022) that essentially involves adding `Let's think step by step` to the original prompt.

In the realm of language models, few-shot learning and prompting play a crucial role. They equip the model with a set of input-output pairs, structured as questions and answers, and subsequently challenge the model to predict the answer for a test-time example. This approach leverages the power of examples to enhance the model's ability to generalize and make accurate predictions.

In [ ]:
// Create a new ChatCompletionsOptions object with a system message and a maximum token count
var chatCompletionsOptions = new ChatCompletionsOptions()
{
    Messages =
    {
        new ChatMessage(ChatRole.System, @"You are an AI assistant that helps to solve math calculations."),
    },
    MaxTokens = 100
};

// Set the chat prompt and add it to the ChatCompletionsOptions object as a user message
string thePrompt = @"
    The odd numbers in this group add up to an even number: 15, 32, 5, 13, 82, 7, 1.
    Let's think step by step, don't answer until you have completed all the steps. 
";
chatCompletionsOptions.Messages.Add(new ChatMessage(ChatRole.User, thePrompt));

// Send the ChatCompletionsOptions object to the OpenAI API and get a response
Response<ChatCompletions> response = client.GetChatCompletions(deploymentOrModelName: deploymentName, chatCompletionsOptions);

// Print the chat response to the console
Console.WriteLine((response.Value.Choices[0].Message.Content));
Console.WriteLine();